# Edición de usuario y avatar

In [ ]:
{% extends "AppCoder/padre.html" %}

{% load static %}

{% block contenidoQueCambia %}

    <!--Aquí va lo que cambia, y lo asociado a está vista :) -->
    <h1>Formulario - Editar Perfil</h1>
    <h1>De {{usuario}}</h1>
    

    <form action="" method="POST">{% csrf_token %}

        <!-- Acá está la magia de Django-->
        
        <table>

            {{ miFormulario }}

        </table>

        <input type="submit", value="Enviar">

    </form>  

{% endblock %}


In [ ]:
from AppCoder.forms import CursoFormulario, ProfesorFormulario, UserRegisterForm, UserEditForm

# Vista de editar el perfil
@login_required
def editarPerfil(request):


    usuario = request.user


    if request.method == 'POST':


        miFormulario = UserEditForm(request.POST, instance=request.user)


        if miFormulario.is_valid():
            miFormulario.save()


            return render(request, "AppCoder/inicio.html")


    else:


        miFormulario = UserEditForm(instance=request.user)


    return render(request, "AppCoder/editar_perfil.html", {"miFormulario": miFormulario, "usuario": usuario})

In [ ]:
path('editarPerfil', views.editarPerfil, name="EditarPerfil"), 

In [ ]:
class UserEditForm(UserChangeForm):
    
    password = None
    email = forms.EmailField(label="Ingrese su email:")
    last_name = forms.CharField(label='Apellido')
    first_name = forms.CharField(label='Nombre')


    class Meta:
        model = User
        fields = ['email', 'last_name', 'first_name']

In [ ]:
from django.contrib.auth.views import PasswordChangeView

In [ ]:
class CambiarContrasenia(LoginRequiredMixin, PasswordChangeView):
    template_name = 'AppCoder/cambiar_contrasenia.html'
    success_url = reverse_lazy('EditarPerfil')

In [ ]:
{% extends "AppCoder/padre.html" %}


{% load static %}


{% block contenidoQueCambia %}
<h1>Cambiar contrasenia</h1>
<form action='' method="POST">
    {% csrf_token %}
    <table>
        {{form}}
    </table>
    <button>Cambiar</button>
</form>


{% endblock %}

In [ ]:
path('cambiarContrasenia', views.CambiarContrasenia.as_view(), name="CambiarContrasenia"),

In [ ]:
<button type='button'>
            <a class="navbar-brand" href="{% url 'CambiarContrasenia' %}">Cambiar contrasenia</a>
        </button>

In [ ]:
{% if not request.user.is_authenticated %}
                <a class="btn btn-primary" href="{% url 'Login' %}">Login</a>
                <a class="btn btn-primary" href="{% url 'Register' %}">Registro</a>
                {% else %}
                <a class="navbar-brand" href="#">{{request.user.username}}</a>
                <a class="btn btn-primary" href="{% url 'Logout' %}">Logout</a>
                {% endif %}

In [ ]:
from django.contrib.auth.models import User
class Avatar(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE)
    imagen = models.ImageField(upload_to='avatares', null=True, blank = True)
    def __str__(self):
        return f"{self.user} - {self.imagen}"


In [ ]:
import os

MEDIA_URL = '/media/'
MEDIA_ROOT = os.path.join(BASE_DIR, 'media')

In [ ]:
# Para las imagenes
from django.conf import settings
from django.conf.urls.static import static

In [ ]:
urlpatterns+= static(settings.MEDIA_URL, document_root = settings.MEDIA_ROOT)

In [ ]:
admin.site.register(models.Avatar)

In [ ]:
      {% if request.user.avatar %}
                    <img src="{{request.user.avatar.imagen.url}}" alt="" height="40px">
                {% endif %}

In [ ]:
class UserEditForm(UserChangeForm):
   
    password = None
    email = forms.EmailField(label="Ingrese su email:")
    last_name = forms.CharField(label='Apellido')
    first_name = forms.CharField(label='Nombre')
    imagen = forms.ImageField(label="Avatar", required=False)


    class Meta:
        model = User
        fields = ['email', 'last_name', 'first_name', 'imagen']

In [ ]:
<form action="" method="POST" enctype="multipart/form-data">

In [ ]:
# Vista de editar el perfil
@login_required
def editarPerfil(request):


    usuario = request.user


    if request.method == 'POST':


        miFormulario = UserEditForm(request.POST, request.FILES, instance=request.user)


        if miFormulario.is_valid():
           
            if miFormulario.cleaned_data.get('imagen'):
                usuario.avatar.imagen = miFormulario.cleaned_data.get('imagen')
                usuario.avatar.save()
           
            miFormulario.save()


            return render(request, "AppCoder/inicio.html")


    else:


        miFormulario = UserEditForm(initial={'imagen': usuario.avatar.imagen}, instance=request.user)


    return render(request, "AppCoder/editar_perfil.html", {"miFormulario": miFormulario, "usuario": usuario})

In [ ]:
<a class="navbar-brand" href="{% url 'EditarPerfil' %}">{{request.user.username}}</a>

# Unit test

In [ ]:
from django.test import TestCase
from AppCoder.models import Profesor
from django.urls import reverse


class EliminarProfesorTest(TestCase):

    def setUp(self):
        self.profesor = Profesor.objects.create(nombre="Juan", apellido="Perez", email="juan@gmail.com", profesion="Ingeniero")
        self.url = reverse('EliminarProfesor', args=[self.profesor.nombre])

    def test_eliminar_profesor(self):
        respuesta = self.client.get(self.url)
        self.assertEqual(respuesta.status_code, 200)
        self.assertTemplateUsed(respuesta, 'AppCoder/leerProfesores.html')
        self.assertQuerysetEqual(Profesor.objects.all(), [])